# 字符级LSTM文本生成

本实验实现基于字符级别的LSTM语言模型，用于自动生成文本。训练数据使用尼采的哲学著作。

**核心思想**：
- 使用滑动窗口从语料库中提取固定长度的字符序列
- LSTM学习字符序列到下一个字符的映射关系
- 通过温度参数控制生成文本的随机性和创造性

In [ ]:
import keras
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子以确保实验可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 下载尼采哲学著作作为训练语料
path = keras.utils.get_file(
    "nietzsche.txt", 
    origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)

# 读取文本并转换为小写，简化字符集
with open(path, encoding='utf-8') as f:
    text = f.read().lower()

print(f'语料库总长度: {len(text)} 个字符')

# 数据预处理：构建训练样本

使用滑动窗口方法从语料库中提取序列对：
- 输入：长度为maxlen的字符序列
- 输出：紧跟在输入序列后的下一个字符

采用One-hot编码将字符转换为向量表示。

In [ ]:
# 超参数设置
maxlen = 60  # 输入序列长度（上下文窗口）
step = 3     # 采样步长，平衡数据量和样本多样性

# 使用滑动窗口提取序列
sentences = []   # 输入序列
next_chars = []  # 目标字符

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print(f"训练样本数量: {len(sentences)}")

# 构建字符表（词汇表）
chars = sorted(list(set(text)))
print(f"词汇表大小: {len(chars)} 个唯一字符")

# 构建字符到索引的映射字典
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

# One-hot编码
print('开始向量化...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_idx[char]] = True
    y[i, char_to_idx[next_chars[i]]] = True

print(f'输入张量形状: {x.shape}')
print(f'输出张量形状: {y.shape}')

# 模型架构

构建单层LSTM网络：
- LSTM层：学习字符序列的长期依赖关系
- Dense层：输出层，生成每个字符的概率分布
- 使用RMSprop优化器和交叉熵损失

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

# 构建模型
model = Sequential([
    layers.LSTM(128, input_shape=(maxlen, len(chars))),
    layers.Dense(len(chars), activation='softmax')
])

# 编译模型
optimizer = RMSprop(learning_rate=0.01)
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

model.summary()

# 温度采样函数

温度参数T调整概率分布的锐度：
- T → 0: 趋向贪婪采样，输出确定性强
- T = 1: 保持原始分布
- T > 1: 增加随机性，输出多样性高

In [ ]:
def sample_next_char(predictions, temperature=1.0):
    """
    基于温度的字符采样函数
    
    参数:
        predictions: 模型输出的概率分布 (vocab_size,)
        temperature: 温度参数，控制采样的随机性
            - T < 1: 更保守，倾向高概率字符
            - T = 1: 标准采样
            - T > 1: 更随机，增加低概率字符的机会
    
    返回:
        采样得到的字符索引
    
    实现细节:
        P_new(i) = exp(log(P(i)) / T) / Σ_j exp(log(P(j)) / T)
    """
    predictions = np.asarray(predictions).astype('float64')
    
    # 温度缩放
    predictions = np.log(predictions + 1e-10) / temperature  # 加小常数避免log(0)
    exp_preds = np.exp(predictions)
    
    # 重新归一化
    predictions = exp_preds / np.sum(exp_preds)
    
    # 多项式采样
    probabilities = np.random.multinomial(1, predictions, 1)
    return np.argmax(probabilities)

# 训练与生成循环

训练策略：
- 每个epoch后生成样本文本，观察模型进化过程
- 使用多个温度参数生成文本，对比不同采样策略的效果
- 随机选择起始序列，增加生成的多样性

In [ ]:
import random
import sys

# 训练参数
NUM_EPOCHS = 60  # 完整训练使用60个epoch
BATCH_SIZE = 128
GENERATION_LENGTH = 400  # 每次生成的字符数
TEMPERATURES = [0.2, 0.5, 1.0, 1.2]  # 测试不同温度参数

for epoch in range(1, NUM_EPOCHS + 1):
    print(f'\n{"="*60}')
    print(f'Epoch {epoch}/{NUM_EPOCHS}')
    print(f'{"="*60}\n')
    
    # 训练一个epoch
    history = model.fit(x, y, batch_size=BATCH_SIZE, epochs=1)
    
    # 随机选择起始序列
    start_index = random.randint(0, len(text) - maxlen - 1)
    seed_text = text[start_index: start_index + maxlen]
    
    print(f'\n种子文本: "{seed_text}"\n')
    
    # 使用不同温度参数生成文本
    for temperature in TEMPERATURES:
        print(f'\n--- 温度 T={temperature} ---')
        sys.stdout.write(seed_text)
        
        generated_text = seed_text
        for i in range(GENERATION_LENGTH):
            # 将当前文本向量化
            sampled_sequence = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled_sequence[0, t, char_to_idx[char]] = 1.
            
            # 预测下一个字符的概率分布
            predictions = model.predict(sampled_sequence, verbose=0)[0]
            
            # 采样下一个字符
            next_idx = sample_next_char(predictions, temperature)
            next_char = idx_to_char[next_idx]
            
            # 更新生成文本（滑动窗口）
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        
        print()  # 换行

print('\n训练完成！')